# Random Forest Model for Occupancy Prediction in an Office Building

## Import Modules

In [94]:
## Data Analytics

import pandas as pd
import numpy as np
import datetime
import random


## Plots

# Import matplotlib and seaborn for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style for plots
plt.style.use('fivethirtyeight')
#plt.rc("font", size=14)
import seaborn as sns
sns.set_theme(style='whitegrid', font='Arial', rc={'figure.figsize':(10,5),
            'font.size':14,
            'axes.titlesize':16,
            'axes.labelsize':15,
            'xtick.labelsize': 12,
            'ytick.labelsize': 12,
            'legend.fontsize': 13},color_codes=True)
#sns.set(style="white")
#sns.set(style="whitegrid", color_codes=True)
#sns.set(rc={'figure.figsize':(20, 10)})
# Pydot is used for visualization
import pydot


## Machine Learning

# Skicit-learn
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Cross Validation
from sklearn.model_selection import TimeSeriesSplit
# Import the model that is used - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier 
# Import tools needed for visualization
from sklearn.tree import export_graphviz
# Import function to calculate accuracy
from sklearn.metrics import accuracy_score
# Confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
# Precision and recall
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
# Import autocorrelation function
from statsmodels.graphics.tsaplots import plot_acf
# Import Matthews Correlation Coefficient
from sklearn.metrics import matthews_corrcoef



# CSV
import csv

## Functions

In [95]:
# Function 1
def predict_with_model(fitted_model, X_val, y_val, output=True):
    '''
    Function to make a prediction based on a trained model, returns metrics for classification
    
    Inputs: 
    fitted_model: On a training set fitted model to use for prediction 
    X_val (np.array): numpy array with feature vectors of validation (or test) data
    y_val (np.array): numpy array with true labels of validation(or test) data
    output (bool): if output should be printed or not, default is True
   
    Outputs:
    y_pred (np.array): Prediction for feature vectors of validation (or test) data of the model
    metrics_dict (dict): Dictionary with metrics for classification
    accuracy (float): Accuracy of the prediction
    precision (float): Precision of the prediction
    recall (float): Recall of the prediction
    f1 (float): F1-Score of the prediction
    mcc (float): Matthews Correlation Coefficient of the prediction
    '''  
    
    # Use the trained model to make predictions on the validation (or test) set
    y_pred = fitted_model.predict(X_val)
    
    ## Metrics for classification
    # Accuracy
    accuracy = accuracy_score(y_val, y_pred)
    # Precision
    precision = precision_score(y_val, y_pred)
    # Recall
    recall = recall_score(y_val, y_pred)
    # F1-Score
    f1 = f1_score(y_val, y_pred)
    # MCC
    mcc = matthews_corrcoef(y_val, y_pred)
    
        
    metrics_dict = {'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                   }
    
    if output:
        print('Mean of true labels:', round(np.mean(y_val), 2))
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall)
        print('F1-Score:', f1)
        print('MCC:', mcc)
            
    return y_pred, metrics_dict, accuracy, precision, recall, f1, mcc

In [100]:
# Function 6
def calculate_feature_importances(model, features, output=True):
    '''
    Function to calculate feature importances of a random forest model
    
    Inputs:
    model: on a training set fitted and on a test set tested model
    features (lst): list containing the names of the features
    output: print results True or False (True by default)
    
    Outputs:
    importances (lst): list with all importances for the features but not sorted (same order as features in input)
    df_feature_importances (df): Dataframe with features and their corresponding importance sorted from highest to lowest
    '''
    # Get numerical feature importances
    importances = list(model.feature_importances_)
    
    # List of tuples with feature and importance
    feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(feature_names, importances)]
    
    # Sort the feature importances from highest to lowest
    feature_importances = sorted(feature_importances, key = lambda x:
                                x[1], reverse = True)
    
    if output:
        # Print out the feature and importances
        [print('Feature: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]
        
    #df_feature_importances = pd.DataFrame(feature_importances, columns=['feature', 'importance'])
    
    return importances, feature_importances

In [101]:
# Function 7
def plot_roc(model, X_test, y_test):
    '''
    Function to plot Receiver Operating Curve in desired layout
    
    Inputs:
    model: on a training set fitted and on a test set tested model
    X_test (np.array): numpy array with feature vectors of test data
    y_test (np.array): numpy array with true labels of test data
    
    Outputs:
    plot of ROC
    
    '''
    model_roc_auc = roc_auc_score(y_test, model.predict(X_test))
    fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label= 'Random Forest '+model_name+' (area = %0.2f)' % model_roc_auc, color='darkblue')
    plt.plot([0,1], [0,1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic - '+e)
    plt.legend(loc="lower right")

In [102]:
# Function 8
def plot_prc(model, X_test, y_test, y_pred):
    '''
    Function to plot Receiver Operating Curve in desired layout
    
    Inputs:
    model: on a training set fitted and on a test set tested model
    X_test (np.array): numpy array with feature vectors of test data
    y_test (np.array): numpy array with true labels of test data
    y_pred (np.array): numpy array with predictions of test data
    
    Outputs:
    plot of Precision Recall Curve
    '''
    
    average_precision = average_precision_score(y_test, y_pred)
    print('Average precision-recall score: {0:0.4f}'.format(average_precision))
    
    disp = plot_precision_recall_curve(model, X_test, y_test, color='darkblue')
    disp.ax_.set_title('2-class Precision-Recall curve: '
                      'AP={0:0.4f}'.format(average_precision))

In [103]:
# Function 8
def plot_actual_predicted_values(X_test_vis, y_test, y_pred):
    '''
    Function to plot actual and predicted values in desired layout 
    
    Inputs:
    model: on a training set fitted and on a test set tested model
    X_test_vis (np.array): numpy array with feature vectors of test data 
                            including columns of month, day, year, hour, minute, second (not one-hot encoded)
    y_test (np.array): numpy array with true labels of test data
    y_pred (np.array): numpy array with predictions of test data
    
    Outputs:
    plot of actual an predicted values
    
    '''
    # Dates of testing data/predictions
    months = X_test_vis[:, data_list.index('Month')]
    days = X_test_vis[:, data_list.index('Day')]
    years = X_test_vis[:, data_list.index('Year')]
    hours = X_test_vis[:, data_list.index('Hour')]
    minutes = X_test_vis[:, data_list.index('Minute')]
    seconds = X_test_vis[:, data_list.index('Second')]
    
    # List and then convert to datetime object
    test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) + ' ' + str(int(hour)) + ':' + str(int(minute)) + ':' + str(int(second)) 
                                                                                      for year, month, day, hour, minute, second in zip(years, months, days, hours, minutes, seconds)]

    test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in test_dates]

    # Dataframe with true values and dates
    true_data = pd.DataFrame(data = {'date': test_dates, 'actual': y_test})

    # Dataframe with predictions and dates
    predictions_data = pd.DataFrame(data = {'date': test_dates, 'prediction': y_pred})
    
    # Plot the actual values
    plt.figure(figsize=(20,5))
    plt.plot(true_data['date'], true_data['actual'],'darkblue', label = 'tatsächlicher Wert')
    #plt.plot(true_data['date'], true_data['actual'],'darkblue', label = 'actual')
    # Plot the predicted values
    plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label = 'vorhergesagter Wert')
    #plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label = 'prediction')
    plt.xticks(rotation = '60');
    plt.xlim(13330, 13515)
    plt.yticks([0,1])
    plt.legend()
    # Graph labels
    plt.xlabel('Datum'); plt.ylabel('Belegung'); plt.title('Tatsächliche und vorhergesagte Werte - '+e);
    #plt.xlabel('Date'); plt.ylabel('Occupancy'); plt.title('Actual and Predicted Values');
    # Dataframe with test values and dates
    testing_data = pd.DataFrame(data = {'date': test_dates, 'actual': y_test})


## Variables

In [104]:
# a = room number
a = 'E07'
# b = number of lags as input
b = '5'
# c = number of last timestep to predict
c = 36
# t = timesteps for seperate plots
t = [6, 12, 18, 24, 30, 36]
# d = number of timestep to predict
# e = timestep in format 't+x' as string

## Loop

In [105]:
for d in range(1,c+1):
    
      
    # Timestep as string
    e = 't+'+str(d)
    print(e)


    #### IMPORT DATA

    # Read in data and set index
    raw_data = pd.read_csv("\Pre-Processing\data_E07_input_5_output_144.txt", parse_dates=True)
    data = raw_data.copy()
    data['DateTime'] = pd.to_datetime(data['DateTime'])
    data = data.set_index('DateTime')



    #### DATA PREPARATION

    # Drop columns for Second
    data = data.drop('Second_0', axis = 1)

    # Drop columns of future timestamps that should not be used as input for this model
    if d==0:
        for i in range(1,145):
            v = 't+'+str(i)
            data = data.drop(v, axis = 1)
    else:
        for i in range(d+1,145):
            v = 't+'+str(i)
            data = data.drop(v, axis = 1)

        for i in range(1, d):
            v = 't+'+str(i)
            data = data.drop(v, axis = 1)


    # Add columns with year, month, day and weekday name at the end of the dataset for later use of visualization 
    data['Year'] = data.index.year
    data['Month'] = data.index.month
    data['Day'] = data.index.day
    data['Weekday Name'] = data.index.day_name()
    data['Hour'] = data.index.hour
    data['Minute'] = data.index.minute
    data['Second'] = data.index.second

    # Use numpy to convert to arrays
    # Labels are the values we want to predict
    labels = np.array(data[e])

    # Remove the labels from the data
    # axis 1 refers to the columns
    data = data.drop(e, axis = 1)

    # Saving data names for later use
    data_list = list(data.columns)

    # Save copy before transforming to numpy array
    data_copy = data.copy()

    # Convert to numpy array
    data = np.array(data)

    # Extract only the one hot encoded data
    feature_names = [i for i in data_list if i not in ['Year', 'Month','Day','Hour','Minute','Second','Weekday Name']]
    indices = [data_list.index(feature_names[x]) for x in range(0,len(feature_names))]
    data_to_use = data[:, indices]


    ### SPLIT DATASET

    # Split data into training and testing sets using scikit-learn 
    # 25% so that approximately the last 6 months are covered 
    # shuffle = False because of time series data
    train_data, test_data, train_labels, test_labels = train_test_split(data_to_use, labels, test_size = 0.25, shuffle=False)
    train_data_vis, test_data_vis, train_labels_vis, test_labels_vis = train_test_split(data, labels, test_size = 0.25, shuffle=False) 


    #### RANDOM FOREST MODELING

    ### WEIGHTED BASE MODEL

    # Instantiate model with tuned hyperparameter
    rf_hyp = RandomForestClassifier(n_estimators = 100, class_weight={0.0: 4, 1.0: 1}, random_state = 42)
    model_name = 'weighted_base'

    # Train the model on training data
    rf_hyp.fit(train_data, train_labels)

    # Calculate accuracy on the training set
    accuracy_train_rf_hyp = rf_hyp.score(train_data, train_labels)

    # Make predictions on test set and calculate metrics
    predictions_rf_hyp, metrics_rf_hyp, accuracy_rf_hyp, precision_rf_hyp, recall_rf_hyp, f1_rf_hyp, mcc_rf_hyp = predict_with_model(rf_hyp, test_data, test_labels)

    # Calculate feature importances
    importances_rf_hyp, feature_importances_rf_hyp = calculate_feature_importances(rf_hyp, feature_names, output=False)


    ## *Visualizations*

    ## Feature importances

    # adapting for each model
    importances = importances_rf_hyp
    features = feature_names

    # list of x locations for plotting
    x_values = list(range(len(importances)))
    # importance in percentages
    importances_percent = [element*100 for element in importances]
    # Make a bar chart
    plt.figure(figsize=(17,5))
    plt.bar(x_values, importances_percent, orientation = 'vertical', color='darkblue', align='center')
    # Tick Labels for x axis
    plt.xticks(x_values, features, rotation = 'vertical')
    if d==0:
        plt.xlim(-1,91)
    else:
        plt.xlim(-1, 92)
        # Axis labels and title
    plt.ylabel('Einfluss in %'); plt.xlabel('Merkmal'); plt.title('Einfluss der Merkmale auf die Vorhersage - ' + e)
    #plt.ylabel('Importance in %'); plt.xlabel('Feature'); plt.title('Feature Importances - ' + e);

    plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_feature_importances.pdf',bbox_inches='tight', dpi=100)
    plt.close()

    # some seperate plots
    if d in t:
       # *Feature importances*
        # list of x locations for plotting
        x_values = list(range(len(importances)))
        # importance in percentages
        importances_percent = [element*100 for element in importances]
        # Make a bar chart
        plt.figure(figsize=(17,5))
        plt.bar(x_values, importances_percent, orientation = 'vertical', color='darkblue', align='center')
        # Tick Labels for x axis
        plt.xticks(x_values, features, rotation = 'vertical')
        plt.xlim(-1, 92)
        plt.ylim(0, 12)
        # Axis labels and title
        plt.ylabel('Einfluss in %'); plt.xlabel('Merkmal'); plt.title('Einfluss der Merkmale auf die Vorhersage - ' + e)
        #plt.ylabel('Importance in %'); plt.xlabel('Feature'); plt.title('Feature Importances - ' + e);
        plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_feature_importances_additional.pdf',bbox_inches='tight', dpi=100)
        plt.close()
    plt.close()

    ## Actual and predicted values
    plot_actual_predicted_values(test_data_vis, test_labels, predictions_rf_hyp)
    plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_actual_and_predicted_values.pdf',bbox_inches='tight', dpi=100)
    plt.close()

    ## Autocorrelation
    plot_acf(predictions_rf_hyp)
    plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_autocorrelation.pdf',bbox_inches='tight', dpi=100)
    plt.close()

    ## Precision Recall Curve

    plot_prc(rf_hyp, test_data, test_labels, predictions_rf_hyp)
    plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_precision_recall_curve.pdf',bbox_inches='tight', dpi=100)
    plt.close()

    ## ROC
    plot_roc(rf_hyp, test_data, test_labels)
    plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_ROC.pdf',bbox_inches='tight', dpi=100)
    plt.close()

    ## Cumulative Importances    

    # List of features sorted from most to least importance 
    sorted_importances = [importance[1] for importance in feature_importances_rf_hyp]
    sorted_features = [importance[0] for importance in feature_importances_rf_hyp]

    # Cumulative importances
    cumulative_importances = np.cumsum(sorted_importances)
    # percentage of cumulate importance
    cumulative_importances_percent = [element*100 for element in cumulative_importances]
    # Make a line graph
    f = plt.subplots(figsize=(17,5))
    plt.plot(x_values, cumulative_importances_percent, 'darkblue', linestyle=' ', marker='.')
    # Draw line at 95% of importance retained
    plt.hlines(y=95, xmin=-1, xmax=len(sorted_importances), color='darkgrey', linestyles = '-')
    plt.hlines(y=90, xmin=-1, xmax=len(sorted_importances), color='darkgrey', linestyles = '-')
    # Format x ticks and labels
    plt.xticks(x_values, sorted_features, rotation = 'vertical')
    if d==0:
        plt.xlim(-1, 91)
    else:
        plt.xlim(-1, 92)
    # Axis labels and title
    plt.xlabel('Merkmal'); plt.ylabel('Kumulierter Einfluss in %');
    plt.title('Kumulierter Einfluss der Merkmale - '+e)
    #plt.xlabel('Feature'); plt.ylabel('Cumulative Importance in %');
    #plt.title('Cumulative Importances - '+e)
    plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_cumulative_importances.pdf',bbox_inches='tight', dpi=100)
    plt.close()

    # Find number of features for cumulative importance of 95%
    # Add 1 because Python is zero-indexed
    cumulative_importance_95 = (np.where(cumulative_importances > 0.95) [0][0]+1)
    print('Number of features for 95% importance: ',
             cumulative_importance_95)
    # Find number of features for cumulative importance of 90%
    # Add 1 because Python is zero-indexed
    cumulative_importance_90 = (np.where(cumulative_importances > 0.90) [0][0]+1)
    print('Number of features for 90% importance: ',
             cumulative_importance_90)


    ## *Confusion Matrix*

    print(confusion_matrix(test_labels, predictions_rf_hyp))
    # Plot confusion matrix
    np.set_printoptions(precision=2)


    #titles_options = [("Konfusionsmatrix ohne Normalisierung", None),
    #                     ("Normalisierte Konfusionsmatrix", 'true')]
    titles_options = [("Confusion matrix without normalization", None),
                         ("Normalized confusion matrix", 'true')]
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(rf_hyp, test_data, test_labels,
                                        cmap=plt.cm.Blues,
                                        normalize=normalize)
        disp.ax_.set_title(title)
        plt.savefig('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_forecast_'+e+'_'+title+'.pdf',bbox_inches='tight', dpi=100)
        plt.close()
    plt.close()


    ## *Save results*

    # Format for saving importances
    header = []
    for k in ['1st Feature', '1st Importance', '2nd Feature', '2nd Importane', '3rd Feature', '3rd Importance']:
        header.append(k)

    for i in range (4,95):
        s = str(i)
        k = s+'th'
        header.append(k+' Feature')
        header.append(k+' Importance')

    header.insert(0, 'Timestep')

    lst1 = [k for (k, v) in feature_importances_rf_hyp]
    lst2 = [v for (k, v) in feature_importances_rf_hyp]

    line = [x for y in zip(lst1, lst2) for x in y]
    line.insert(0, e)

    # Save
    if d==1:
            # Creating csv file with results
            with open('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_results.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Timestep','Accuracy - Training Set', 'Accuracy - Test Set','Precision', 'Recall','F1-Score', 'Matthews Correlation Coefficient', '95%', '90%'])
                writer.writerow([e, accuracy_train_rf_hyp, accuracy_rf_hyp, precision_rf_hyp, recall_rf_hyp, f1_rf_hyp, mcc_rf_hyp, cumulative_importance_95, cumulative_importance_90])
            # Creating csv file with results of importances
            with open('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_feature_importances.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(header)
                writer.writerow(line)
    else:    
            # Appending results for model to csv file
            with open('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_results.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([e, accuracy_train_rf_hyp, accuracy_rf_hyp, precision_rf_hyp, recall_rf_hyp, f1_rf_hyp, mcc_rf_hyp, cumulative_importance_95, cumulative_importance_90])
            # Appending results of importances to csv file
            with open('Random_Forest_'+a+'_'+model_name+'_input_'+b+'_feature_importances.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(line)

t+1
Mean of true labels: 0.17
Accuracy: 0.972518457752256
Precision: 0.9162779278173336
Recall: 0.9189854715587293
F1-Score: 0.917629702483403
MCC: 0.9011392564788097
Average precision-recall score: 0.8555
Number of features for 95% importance:  46
Number of features for 90% importance:  25
[[19978   341]
 [  329  3732]]
Mean of true labels: 0.17
Accuracy: 0.972149302707137
Precision: 0.9116358325219085
Recall: 0.9221866535336124
F1-Score: 0.9168808911739504
MCC: 0.9001752094625649
Average precision-recall score: 0.8537
Number of features for 95% importance:  56
Number of features for 90% importance:  39
[[19956   363]
 [  316  3745]]
Mean of true labels: 0.17
Accuracy: 0.9710828547990156
Precision: 0.9133004926108375
Recall: 0.9130755971435607
F1-Score: 0.9131880310306613
MCC: 0.8958401733119499
[[19967   352]
 [  353  3708]]
Mean of true labels: 0.17
Accuracy: 0.9629614438063987
Precision: 0.8963353413654619
Recall: 0.8793400640236395
F1-Score: 0.8877563704164076
MCC: 0.8656371112164

[[19413   899]
 [ 2730  1338]]
t+9
Mean of true labels: 0.17
Accuracy: 0.9150123051681707
Precision: 0.7599583441812028
Recall: 0.7173752764807078
F1-Score: 0.7380530973451327
MCC: 0.6877883113510006
Average precision-recall score: 0.5923
Number of features for 95% importance:  70
Number of features for 90% importance:  57
[[19389   922]
 [ 1150  2919]]
Mean of true labels: 0.17
Accuracy: 0.9198113207547169
Precision: 0.7325120985481742
Recall: 0.8183828950602113
F1-Score: 0.7730702263493906
MCC: 0.7261948219876367
Average precision-recall score: 0.6298
Number of features for 95% importance:  72
Number of features for 90% importance:  62
[[19095  1216]
 [  739  3330]]
Mean of true labels: 0.17
Accuracy: 0.8321985233798195
Precision: 0.35526315789473684
Recall: 0.006635536986974687
F1-Score: 0.013027744270205067
MCC: 0.028248160000075052
[[20262    49]
 [ 4042    27]]
Mean of true labels: 0.17
Accuracy: 0.8449138638228055
Precision: 0.5701754385964912
Recall: 0.28753993610223644
F1-Scor

[[19205  1099]
 [ 2766  1310]]
t+17
Mean of true labels: 0.17
Accuracy: 0.8993027071369976
Precision: 0.7197831978319783
Recall: 0.6514594064262939
F1-Score: 0.6839191451010687
MCC: 0.6253057835393151
Average precision-recall score: 0.5272
Number of features for 95% importance:  75
Number of features for 90% importance:  63
[[19269  1034]
 [ 1421  2656]]
Mean of true labels: 0.17
Accuracy: 0.9075061525840853
Precision: 0.7053651938683498
Recall: 0.7674760853568801
F1-Score: 0.7351110066956419
MCC: 0.6800901753419949
Average precision-recall score: 0.5802
Number of features for 95% importance:  74
Number of features for 90% importance:  65
[[18996  1307]
 [  948  3129]]
Mean of true labels: 0.17
Accuracy: 0.8327317473338802
Precision: 0.0
Recall: 0.0
F1-Score: 0.0
MCC: -0.002870002696239167
[[20302     1]
 [ 4077     0]]
Mean of true labels: 0.17
Accuracy: 0.840935192780968
Precision: 0.5421788893598982
Recall: 0.3137110620554329
F1-Score: 0.39745183343691737
MCC: 0.32885463235504747
[[

C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Anna-Lena\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Mean of true labels: 0.17
Accuracy: 0.832526661197703
Precision: 0.0
Recall: 0.0
F1-Score: 0.0
MCC: 0.0
[[20297     0]
 [ 4083     0]]
Mean of true labels: 0.17
Accuracy: 0.8202214930270714
Precision: 0.4321880650994575
Recall: 0.23414156257653687
F1-Score: 0.3037331215250199
MCC: 0.22394196906030167
[[19041  1256]
 [ 3127   956]]
t+24
Mean of true labels: 0.17
Accuracy: 0.8872026251025431
Precision: 0.697014484185634
Recall: 0.5775165319617928
F1-Score: 0.6316635413876239
MCC: 0.5692462400269303
Average precision-recall score: 0.4733
Number of features for 95% importance:  78
Number of features for 90% importance:  66
[[19272  1025]
 [ 1725  2358]]
Mean of true labels: 0.17
Accuracy: 0.8939704675963905
Precision: 0.6711608775137111
Recall: 0.7193240264511389
F1-Score: 0.6944083224967491
MCC: 0.6309130719368385
Average precision-recall score: 0.5298
Number of features for 95% importance:  76
Number of features for 90% importance:  67
[[18858  1439]
 [ 1146  2937]]
Mean of true labels: 

Mean of true labels: 0.17
Accuracy: 0.8209187858900738
Precision: 0.28912071535022354
Recall: 0.04751408278226794
F1-Score: 0.08161548169962136
MCC: 0.05480692984140902
[[19820   477]
 [ 3889   194]]
Mean of true labels: 0.17
Accuracy: 0.8080803937653814
Precision: 0.2772795216741405
Recall: 0.09086456037227529
F1-Score: 0.13687511529238147
MCC: 0.07086354112939201
[[19330   967]
 [ 3712   371]]
t+32
Mean of true labels: 0.17
Accuracy: 0.8841673502871206
Precision: 0.7019570099454604
Recall: 0.5358804800391869
F1-Score: 0.6077777777777778
MCC: 0.5480480776548422
Average precision-recall score: 0.4539
Number of features for 95% importance:  80
Number of features for 90% importance:  70
[[19368   929]
 [ 1895  2188]]
Mean of true labels: 0.17
Accuracy: 0.8916324856439705
Precision: 0.6775067750677507
Recall: 0.6735243693362724
F1-Score: 0.6755097027757307
MCC: 0.6104691492533035
Average precision-recall score: 0.5110
Number of features for 95% importance:  76
Number of features for 90% i

In [106]:
rf_base.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}